In [1]:
import sys
sys.path.append('/share/software/user/open/py-numpy/1.18.1_py36/lib/python3.6/site-packages')
sys.path.append('/share/software/user/open/py-pandas/1.0.3_py36/lib/python3.6/site-packages')

# package provided in uhgg/
sys.path.append('/home/users/zhiru/uhgg-helper')

In [2]:
import numpy as np

In [3]:
import UHGG_utils

In [9]:
accession = 'MGYG-HGUT-02492'  # uhgg accession for E. rectale

# filtered SNVs tables can be computed with uhgg/scripts/filter_snv_table_by_freq.py
idx = 1
freq_range_name = ['big-freq', 'med-freq', 'small-freq'][idx]
freq_range = [[0.175, 0.1925], [0.025, 0.0275], [0.00625, 0.006875]][idx]
print("Using frequency range {}: min {} - max {}".format(freq_range_name, freq_range[0], freq_range[1]))
filepath = '/home/groups/bhgood/uhgg/filtered_snvs/{}_filtered_snvs_{}.tsv'.format(accession, freq_range_name)

# load the filter 
genome_mask = UHGG_utils.get_non_redundant_genome_mask_by_accession(accession)
true_zeros, true_ones, locations, mut_types = UHGG_utils.load_biallelic_snvs_annotated(filepath, genome_mask)

Using frequency range med-freq: min 0.025 - max 0.0275


/share/software/user/open/py-jupyter/1.0.0_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [10]:
%%time
import itertools as it
n00s = np.dot(true_zeros, true_zeros.T)  # using dot product to count the number of genomes with 00 genotype
n10s = np.dot(true_ones, true_zeros.T)
n11s = np.dot(true_ones, true_ones.T)

all_res = []
for i, j in it.combinations(range(true_zeros.shape[0]), 2):
    ell = np.abs(locations[i] - locations[j])
    n00 = n00s[i, j]
    n10 = n10s[i, j]
    n01 = n10s[j, i]
    n11 = n11s[i, j]
    # res = (n11, n10, n01, n00, str(ell), contig, str(gene_id))
    # shortened output to save memory
    res = (n11, n10, n01, n00, ell)
    all_res.append(res)

CPU times: user 1min 5s, sys: 290 ms, total: 1min 5s
Wall time: 1min 5s


In [11]:
ns = np.array(all_res)

n11s = ns[:, 0]
n10s = ns[:, 1]
n01s = ns[:, 2]
n00s = ns[:, 3]
ells = ns[:, 4]
ntots = n11s+n10s+n01s+n00s

max_ntot = ntots.max()

good_idxs = (ntots>0.95*max_ntot)
print("max_ntot", max_ntot)
print("Kept", good_idxs.sum()*1.0/len(good_idxs))

n11s = n11s[good_idxs]
n10s = n10s[good_idxs]
n01s = n01s[good_idxs]
n00s = n00s[good_idxs]
ntots = ntots[good_idxs]
ells = ells[good_idxs]

min_ntot = ntots.min()

f11s = n11s*1.0/ntots
f10s = n10s*1.0/ntots
f01s = n01s*1.0/ntots
f00s = n00s*1.0/ntots

fAs = f11s+f10s
fBs = f11s+f01s

nAs = n11s+n10s
nBs = n11s+n01s

max_ntot 4737
Kept 0.04888714347861272


In [12]:
ellmin,ellmax = 1e6, 2e6

good_idxs = (ells>ellmin)*(ells<ellmax)

lambs = f11s[good_idxs] * f10s[good_idxs] * f01s[good_idxs] * f00s[good_idxs]
lambs /= fAs[good_idxs]**2 * fBs[good_idxs]**2 * (1-fAs[good_idxs])**2 * (1-fBs[good_idxs])**2

In [13]:
# saving intermediate data for plotting
np.save('LEs_uhgg_{}'.format(freq_range_name), lambs)